In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from util_visualization import plot_feature_importances, plot_roc_curve
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, mean_absolute_percentage_error, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve
from pylab import rcParams
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import pickle
import importlib
import sys
import joblib
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import glob

#For NN
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import GridSearchCV
import numpy as np
import sys
sys.setrecursionlimit(10**6)  # Set the recursion limit to a higher value

# Import Data

In [6]:
#Import Dataset
labeled_index = pd.read_csv("../data/labeled_data/quarterly_labeled_features_standardized.csv")
labeled_index.set_index(labeled_index['date'], inplace=True)
quarters = np.sort(labeled_index.index.unique())
print(labeled_index)

#Features Columns
features_columns = list(labeled_index.columns)
columns_to_remove = ['index', 'crash_label', 'date', 'Quarter']
features_columns = [column for column in features_columns if column not in columns_to_remove]
print(features_columns)

            volatility      index  crash_label  price_change  volume_change  \
date                                                                          
1998-03-31   -0.628541  000001.SS            0      0.187709      -0.127147   
1998-06-30   -0.628541  000001.SS            0      0.504913      -0.127147   
1998-09-30   -0.625680  000001.SS            1     -0.795446      -0.127147   
1998-12-31   -0.631403  000001.SS            0     -0.843210      -0.127147   
1999-03-31   -0.629515  000001.SS            0     -0.082838      -0.127147   
...                ...        ...          ...           ...            ...   
2018-12-31   -0.493234      ^SSMI            0     -0.800378      -0.127147   
2019-03-31   -0.529809      ^SSMI            0      0.914383      -0.127147   
2019-06-30   -0.458321      ^SSMI            0      0.217264      -0.127147   
2019-09-30   -0.388212      ^SSMI            0     -0.010617      -0.127147   
2019-12-31   -0.351813      ^SSMI            0      

# Build the model

In [7]:
# Define your neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Custom estimator class for grid search compatibility
class NeuralNetworkEstimator(nn.Module, GridSearchCV):
    def __init__(self, input_size, hidden_size, lr):
        nn.Module.__init__(self)
        GridSearchCV.__init__(self, estimator=self, param_grid={})  # Initialize GridSearchCV

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lr = lr
        self.model = NeuralNetwork(input_size, hidden_size)

    def fit(self, X, y):
        # Convert data to PyTorch tensors
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.long)

        # Create a DataLoader for training set
        train_dataset = TensorDataset(X_tensor, y_tensor)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

        # Train the neural network model
        for epoch in range(10):  # Adjust the number of epochs as needed
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

    def predict(self, X):
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32)
            outputs = self.model(X_tensor)
            _, predicted = torch.max(outputs, 1)
            return predicted.numpy()

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
evaluation = []
input_size = len(features_columns)


for train_index, test_index in tscv.split(quarters):
    train_quarters, test_quarters = quarters[train_index], quarters[test_index]
    train = labeled_index.loc[train_quarters]
    test = labeled_index.loc[test_quarters]
    X_train = train[features_columns]
    y_train = train['crash_label']
    X_test = test[features_columns]
    y_test = test['crash_label']

    # Oversample the minority class (1) using SMOTE
    oversampler = SMOTE(sampling_strategy=0.5, random_state=42)
    X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

    # Undersample the majority class (0) using RandomUnderSampler
    undersampler = RandomUnderSampler(sampling_strategy=1.0, random_state=42)
    X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_oversampled, y_train_oversampled)
    
    input_size = X_train_resampled.shape[1]  # Number of input features
    hidden_size = 64  # Number of neurons in the hidden layer
    lr = 0.001  # Learning rate for the neural network
    
    # Perform grid search to find optimal hyperparameters
    # Define the hyperparameter grid for the neural network
    param_grid = {'hidden_size': [64, 128, 256],'lr': [0.001, 0.01, 0.1]}
    grid_search = GridSearchCV(NeuralNetworkEstimator(input_size, hidden_size, lr), param_grid, verbose=2)    
    grid_search.fit(X_train_resampled, y_train_resampled)
    best_params = grid_search.best_params_

    # Create a new instance of your neural network model with optimal hyperparameters
    model = NeuralNetworkEstimator(input_size, hidden_size, lr)

    # Fit the model
    model.fit(X_train_resampled, y_train_resampled)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    decision_values = model.decision_function(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    conf_matrix = confusion_matrix(y_test, y_pred, labels=[0, 1])
    
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, decision_values)
    fpr, tpr, thresholds = roc_curve(y_test, decision_values)
    
    evaluation_result = {
        'Train_Start': pd.to_datetime(train_quarters).min(),
        'Train_End': pd.to_datetime(train_quarters).max(),
        'Test_Start': pd.to_datetime(test_quarters).min(),
        'Test_End': pd.to_datetime(test_quarters).max(),
        'Confusion_Matrix': conf_matrix,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Accuracy': accuracy,
        'RMSE': rmse,
        'MAPE': mape,
        'AUC-ROC': auc_roc,
        'False Positive Rate': fpr,
        'True Positive Rate': tpr
    }

    # Calculate feature importance
    feature_importance = np.abs(model.dual_coef_).sum(axis=0)
    for name, importance in zip(features_columns, feature_importance):
        evaluation_result[f"{name}_importance"] = importance

    # Append result to evaluation
    evaluation.append(evaluation_result)

    print("Prediction model trained")